# Foundations
While this notebook is called Foundations, it's more appropriately coined as an overview/refresher on CS topics like callbacks.

In [1]:
import torch
import matplotlib.pyplot as plt
import random

## Callbacks

### Callbacks as GUI events

In [2]:
import ipywidgets as widgets

In [3]:
w = widgets.Button(description="Click me")

hi
hi
hi


In [4]:
w

Button(description='Click me', style=ButtonStyle())

In [5]:
def f(o): print('hi')

In [6]:
w.on_click(f)

The code above is an example of callbacks, more specifically, events. This is a common concept in Javascript, especially for front end dev.

### Creating your own callback

In [7]:
from time import sleep

In [8]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i**2
        sleep(1)
    return res

In [9]:
slow_calculation()

30

Now it's time to add a callback to our `slow_calculation` function.

In [10]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i**2
        sleep(1)
        if cb: cb(i)
    return res

In [11]:
def show_progress(epoch):
    print(f"Awesome! We've finished epoch {epoch}!")

In [12]:
slow_calculation(show_progress)

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

### Lambdas and partials
Now we can use Lambdas and partials to further improve our code. Firstly, we can use `lambda` directly without defining the callback function elsewhere.

In [14]:
slow_calculation(lambda o: print(f"Awesome! We have finished epoch {o}!"))

Awesome! We have finished epoch 0!
Awesome! We have finished epoch 1!
Awesome! We have finished epoch 2!
Awesome! We have finished epoch 3!
Awesome! We have finished epoch 4!


30

In [15]:
def show_progress(exclamation, epoch):
    print(f"{exclamation}! We've finished epoch {epoch}!")

In [16]:
slow_calculation(lambda o: show_progress("Wow", o))

Wow! We've finished epoch 0!
Wow! We've finished epoch 1!
Wow! We've finished epoch 2!
Wow! We've finished epoch 3!
Wow! We've finished epoch 4!


30

We use inner functions for encapsulation and modularity. For example, we can use different `exclamation` parameters at different parts of the code.

In [17]:
def make_show_progress(exclamation):
    def _inner(epoch): print(f"{exclamation}! We have finished epoch {epoch}!")
    return _inner

In [18]:
slow_calculation(make_show_progress("Ayo"))

Ayo! We have finished epoch 0!
Ayo! We have finished epoch 1!
Ayo! We have finished epoch 2!
Ayo! We have finished epoch 3!
Ayo! We have finished epoch 4!


30

Next up, we will use partials.

In [19]:
from functools import partial

The code below uses `partial` to pass "Ok I guess" as the first parameter to `show_progress`.

In [20]:
slow_calculation(partial(show_progress, "Ok I guess"))

Ok I guess! We've finished epoch 0!
Ok I guess! We've finished epoch 1!
Ok I guess! We've finished epoch 2!
Ok I guess! We've finished epoch 3!
Ok I guess! We've finished epoch 4!


30

In [21]:
f2 = partial(show_progress, "Ok I guess")

### Callbacks as callable classes

In [22]:
class ProgressShowingCallback():
    def __init__(self, exclamation="Awesome"):
        self.exclamation = exclamation

    def __call__(self, epoch):
        print(f"{self.exclamation}! We have finished epoch {epoch}!")

In [23]:
cb = ProgressShowingCallback("Wow, fantastic baby")

In [24]:
slow_calculation(cb)

Wow, fantastic baby! We have finished epoch 0!
Wow, fantastic baby! We have finished epoch 1!
Wow, fantastic baby! We have finished epoch 2!
Wow, fantastic baby! We have finished epoch 3!
Wow, fantastic baby! We have finished epoch 4!


30

We see that when we call `cb` as a callback function, the epoch number is passed into it.

### Multiple callback funcs; `*args` and `**kwargs`

In [25]:
def f(*a, **b):
    print(f"args: {a}; kwargs: {b}")

In [26]:
f(3, 'a', thing1="hello")

args: (3, 'a'); kwargs: {'thing1': 'hello'}


In [27]:
def g(a,b,c=0): print(a, b, c)

In [28]:
args = [1,2]
kwargs = {'c':3}
g(*args, **kwargs)

1 2 3


In [29]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i**2
        sleep(1)
        if cb: cb.after_calc(i, val=res)
    return res

In [30]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs):
        print(f"About to start")
    def after_calc(self, *args, **kwargs):
        print(f"Done step")

In [34]:
slow_calculation(PrintStepCallback)

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In [35]:
class PrintStatusCallback():
    def __init__(self): pass
    def before_calc(self, epoch, **kwargs):
        print(f"About to start epoch {epoch}")

    def after_calc(self, epoch, val, **kwargs):
        print(f"After epoch {epoch}: {val}")

In [37]:
slow_calculation(PrintStatusCallback())

About to start epoch 0
After epoch 0: 0
About to start epoch 1
After epoch 1: 1
About to start epoch 2
After epoch 2: 5
About to start epoch 3
After epoch 3: 14
About to start epoch 4
After epoch 4: 30


30

We see that Python automatically passes in the `epoch` aed `val` to the callback function.

### Modifying behavior

In [38]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb, 'before_calc'):
            cb.before_calc(i)

        res += i**2
        sleep(1)
        if cb and hasattr(cb, 'after_calc'):
            if cb.after_calc(i, res):
                print("stopping early")
                break

    return res

In [39]:
class PrintAfterCallback():
    def after_calc(self, epoch, val):
        print(f"After epoch {epoch}: {val}")
        if val>10: return True

In [40]:
slow_calculation(PrintAfterCallback())

After epoch 0: 0
After epoch 1: 1
After epoch 2: 5
After epoch 3: 14
stopping early


14

In [51]:
class SlowCalculator():
    def __init__(self, cb=None):
        self.cb, self.res = cb, 0

    def callback(self, cb_name, *args):
        if not self.cb: return
        cb = getattr(self.cb, cb_name, None)
        if cb: return cb(self, *args)

    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i**2
            sleep(1)
            if self.callback('after_calc', i):
                print("stopping early")
                break

In [52]:
class ModifyingCallback():
    def after_calc(self, calc, epoch):
        print(f"After {epoch}: {calc.res}")
        if calc.res>10: return True
        if calc.res<3: calc.res *= 2

In [53]:
calculator = SlowCalculator(ModifyingCallback())

In [54]:
calculator.calc()

After 0: 0
After 1: 1
After 2: 6
After 3: 15
stopping early


## `__dunder__`

Anything that looks like __this__ is, in some way, special. Python, or some library, can define some functions that they will call at certain documented times. For instance, when your class is setting up a new object, python will call __init__. These are defined as part of the python data model.

For instance, if python sees +, then it will call the special method __add__. If you try to display an object in Jupyter (or lots of other places in Python) it will call __repr__.